In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

import string, re
import nltk
nltk.download('wordnet')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
import spacy

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mukhtar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mukhtar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ImportError: No module named spacy

In [ ]:
tweets_df = pd.read_csv('file.csv')

In [ ]:
tweets_df.columns

In [12]:
all_tweets = tweets_df[:190000]
caa_tweets = all_tweets.tweet
hashtags = all_tweets.hashtags
date = all_tweets.date
retweets_count = all_tweets.retweets_count
likes_count = all_tweets.likes_count
useful_tweets = list(zip(caa_tweets,date, hashtags, retweets_count, likes_count))

In [13]:
#functuion to seperate hindi and others
def separate_english_hindi_tweets(tweets):
    etweets = []
    htweets= []
    
    for t, tweet_tup in enumerate(tweets):
        e_flag = 1
        tweet = tweet_tup[0]
        for c in tweet:
            if c == "\n":
                continue
            if ord(c) > 31 and ord(c) < 127:
                continue
            elif ord(c) > 2300 and ord(c) < 2400: # hindi
                e_flag = 0
                htweets.append(tweet_tup)
                break
            else:
                continue
        if e_flag == 1:
            etweets.append(tweet_tup)
            
    return etweets, htweets

In [14]:
english_tweets = []
hindi_tweets = []
english_tweets, hindi_tweets = separate_english_hindi_tweets(useful_tweets)

In [48]:
# Separate English from others

etweets = []
otweets= []
    
for t, etup in enumerate(english_tweets):
    e_flag = 1
    tweet = etup[0]
    for c in tweet:
        if c == "\n":
            continue
        if ord(c) > 31 and ord(c) < 250:
            continue
        elif ord(c) > 8200 and ord(c) < 8400: #special punctuations
            continue
        elif ord(c) > 9000: #smileys
            continue
        else:
#             print(c, ord(c))
            e_flag = 0
            break
    if e_flag == 1:
        etweets.append(etup)
    else:
        otweets.append(etup)

In [49]:
etweets[1]

('Yet another reason why India needs #CAA: \n\nHindus Beaten by Pakistani Police for Hoisting Saffron Flag in Their Own Home. Video Published to Cower Other Hindus into Submission!\n\n https://www.youtube.com/watch?v=lTQxDeBmCyI\xc2\xa0\xe2\x80\xa6\n@MEAIndia @Swamy39 @blsanthosh @davidfrawleyved @MODIfiedVikas @ShefVaidya',
 '2020-01-08',
 "['#caa']",
 0L,
 0L)

In [21]:
retweet_sorted_tweets = sorted(etweets, key=lambda x: x[3], reverse=True)

In [50]:
retweet_sorted_tweets[:10]

[('#CAA + #NRC + more this Sunday on @patriotact pic.twitter.com/AIoAub8Fwu',
  '2019-12-20',
  "['#caa', '#nrc']",
  17739L,
  42902L),
 ('\xe0\xa4\xac\xe0\xa4\xa1\xe0\xa4\xbc\xe0\xa5\x80 \xe0\xa4\xb8\xe0\xa4\x82\xe0\xa4\x96\xe0\xa5\x8d\xe0\xa4\xaf\xe0\xa4\xbe \xe0\xa4\xae\xe0\xa5\x87\xe0\xa4\x82 \xe0\xa4\xb2\xe0\xa5\x8b\xe0\xa4\x97 \xe0\xa4\xaa\xe0\xa5\x82\xe0\xa4\x9b \xe0\xa4\xb0\xe0\xa4\xb9\xe0\xa5\x87 \xe0\xa4\xb9\xe0\xa5\x88\xe0\xa4\x82-\xe0\xa4\x9c\xe0\xa5\x8b \xe0\xa4\xb2\xe0\xa5\x8b\xe0\xa4\x97 #CAA \xe0\xa4\x95\xe0\xa5\x87 \xe0\xa4\xb5\xe0\xa4\xbf\xe0\xa4\xb0\xe0\xa5\x8b\xe0\xa4\xa7 \xe0\xa4\xae\xe0\xa5\x87\xe0\xa4\x82 \xe0\xa4\xb9\xe0\xa5\x88\xe0\xa4\x82 \xe0\xa4\xb5\xe0\xa5\x8b \xe0\xa4\xb8\xe0\xa4\xac \xe0\xa4\xb8\xe0\xa4\xa1\xe0\xa4\xbc\xe0\xa4\x95\xe0\xa5\x8b\xe0\xa4\x82 \xe0\xa4\xaa\xe0\xa4\xb0 \xe0\xa4\xa6\xe0\xa4\xbf\xe0\xa4\x96  \xe0\xa4\xb0\xe0\xa4\xb9\xe0\xa5\x87 \xe0\xa4\xb9\xe0\xa5\x88\xe0\xa4\x82 \xe0\xa4\xb2\xe0\xa5\x87\xe0\xa4\x95\xe0\xa4\xbf\xe0\xa4\xa8 #CAA 

In [33]:
UPPERLETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
LETTERS_AND_SPACE = UPPERLETTERS + UPPERLETTERS.lower() + ' \t\n'
def loadDictionary():
    dictionaryFile = open('dictionary.txt')
    englishWords = {}
    for word in dictionaryFile.read().split('\n'):
        englishWords[word] = None
        dictionaryFile.close()
        return englishWords

In [56]:
def loadDictionary():
    f = open('dictionary.txt', 'rU')
    for line in f:
        print line
    f.close()

In [57]:
ENGLISH_WORDS = loadDictionary()
def get_English_Count(etweets):
    etweets = etweets.upper()
    etweets = removeNonLetters(etweets)
    possibleWords = etweets.split()
    if possibleWords == []:
        return 0.0 # no words at all, so return 
    matches = 0
    for word in possibleWords:
        if word in ENGLISH_WORDS:
            matches += 1
    return float(matches) / len(possibleWords)


AARHUS

AARON

ABABA

ABACK

ABAFT

ABANDON

ABANDONED

ABANDONING

ABANDONMENT

ABANDONS

ABASE

ABASED

ABASEMENT

ABASEMENTS

ABASES

ABASH

ABASHED

ABASHES

ABASHING

ABASING

ABATE

ABATED

ABATEMENT

ABATEMENTS

ABATER

ABATES

ABATING

ABBA

ABBE

ABBEY

ABBEYS

ABBOT

ABBOTS

ABBOTT

ABBREVIATE

ABBREVIATED

ABBREVIATES

ABBREVIATING

ABBREVIATION

ABBREVIATIONS

ABBY

ABDOMEN

ABDOMENS

ABDOMINAL

ABDUCT

ABDUCTED

ABDUCTION

ABDUCTIONS

ABDUCTOR

ABDUCTORS

ABDUCTS

ABE

ABED

ABEL

ABELIAN

ABELSON

ABERDEEN

ABERNATHY

ABERRANT

ABERRATION

ABERRATIONS

ABET

ABETS

ABETTED

ABETTER

ABETTING

ABEYANCE

ABHOR

ABHORRED

ABHORRENT

ABHORRER

ABHORRING

ABHORS

ABIDE

ABIDED

ABIDES

ABIDING

ABIDJAN

ABIGAIL

ABILENE

ABILITIES

ABILITY

ABJECT

ABJECTION

ABJECTIONS

ABJECTLY

ABJECTNESS

ABJURE

ABJURED

ABJURES

ABJURING

ABLATE

ABLATED

ABLATES

ABLATING

ABLATION

ABLATIVE

ABLAZE

ABLE

ABLER

ABLEST

ABLY

ABNER

ABNORMAL

ABNORMALITIES

ABNORMALITY

ABNORMALLY

ABO

AMBIGUITIES

AMBIGUITY

AMBIGUOUS

AMBIGUOUSLY

AMBITION

AMBITIONS

AMBITIOUS

AMBITIOUSLY

AMBIVALENCE

AMBIVALENT

AMBIVALENTLY

AMBLE

AMBLED

AMBLER

AMBLES

AMBLING

AMBROSIAL

AMBULANCE

AMBULANCES

AMBULATORY

AMBUSCADE

AMBUSH

AMBUSHED

AMBUSHES

AMDAHL

AMELIA

AMELIORATE

AMELIORATED

AMELIORATING

AMELIORATION

AMEN

AMENABLE

AMEND

AMENDED

AMENDING

AMENDMENT

AMENDMENTS

AMENDS

AMENITIES

AMENITY

AMENORRHEA

AMERADA

AMERICA

AMERICAN

AMERICANA

AMERICANISM

AMERICANIZATION

AMERICANIZATIONS

AMERICANIZE

AMERICANIZER

AMERICANIZERS

AMERICANIZES

AMERICANS

AMERICAS

AMERICIUM

AMES

AMHARIC

AMHERST

AMIABLE

AMICABLE

AMICABLY

AMID

AMIDE

AMIDST

AMIGA

AMIGO

AMINO

AMISS

AMITY

AMMAN

AMMERMAN

AMMO

AMMONIA

AMMONIAC

AMMONIUM

AMMUNITION

AMNESTY

AMOCO

AMOEBA

AMOEBAE

AMOEBAS

AMOK

AMONG

AMONGST

AMONTILLADO

AMORAL

AMORALITY

AMORIST

AMOROUS

AMORPHOUS

AMORPHOUSLY

AMORTIZE

AMORTIZED

AMORTIZES

AMORTIZING

AMOS

AMOUNT

AMOUNTED

AMOUNTER

AMOUN

ASSURES

ASSURING

ASSURINGLY

ASSYRIA

ASSYRIAN

ASSYRIANIZE

ASSYRIANIZES

ASSYRIOLOGY

ASTAIRE

ASTAIRES

ASTARTE

ASTATINE

ASTER

ASTERISK

ASTERISKS

ASTEROID

ASTEROIDAL

ASTEROIDS

ASTERS

ASTHMA

ASTON

ASTONISH

ASTONISHED

ASTONISHES

ASTONISHING

ASTONISHINGLY

ASTONISHMENT

ASTOR

ASTORIA

ASTOUND

ASTOUNDED

ASTOUNDING

ASTOUNDS

ASTRAL

ASTRAY

ASTRIDE

ASTRINGENCY

ASTRINGENT

ASTROLOGY

ASTRONAUT

ASTRONAUTICS

ASTRONAUTS

ASTRONOMER

ASTRONOMERS

ASTRONOMICAL

ASTRONOMICALLY

ASTRONOMY

ASTROPHYSICAL

ASTROPHYSICS

ASTUTE

ASTUTELY

ASTUTENESS

ASUNCION

ASUNDER

ASYLUM

ASYMMETRIC

ASYMMETRICALLY

ASYMMETRY

ASYMPTOMATICALLY

ASYMPTOTE

ASYMPTOTES

ASYMPTOTIC

ASYMPTOTICALLY

ASYNCHRONISM

ASYNCHRONOUS

ASYNCHRONOUSLY

ASYNCHRONY

ATALANTA

ATARI

ATAVISTIC

ATCHISON

ATE

ATEMPORAL

ATHABASCAN

ATHEISM

ATHEIST

ATHEISTIC

ATHEISTS

ATHENA

ATHENIAN

ATHENIANS

ATHENS

ATHEROSCLEROSIS

ATHLETE

ATHLETES

ATHLETIC

ATHLETICISM

ATHLETICS

ATKINS

ATKINSON

ATLANTA

A


BEHAVES

BEHAVING

BEHAVIOR

BEHAVIORAL

BEHAVIORALLY

BEHAVIORISM

BEHAVIORISTIC

BEHAVIORS

BEHEAD

BEHEADING

BEHELD

BEHEMOTH

BEHEMOTHS

BEHEST

BEHIND

BEHOLD

BEHOLDEN

BEHOLDER

BEHOLDERS

BEHOLDING

BEHOLDS

BEHOOVE

BEHOOVES

BEIGE

BEIJING

BEING

BEINGS

BEIRUT

BELA

BELABOR

BELABORED

BELABORING

BELABORS

BELATED

BELATEDLY

BELAY

BELAYED

BELAYING

BELAYS

BELCH

BELCHED

BELCHES

BELCHING

BELFAST

BELFRIES

BELFRY

BELGIAN

BELGIANS

BELGIUM

BELGRADE

BELIE

BELIED

BELIEF

BELIEFS

BELIES

BELIEVABLE

BELIEVABLY

BELIEVE

BELIEVED

BELIEVER

BELIEVERS

BELIEVES

BELIEVING

BELITTLE

BELITTLED

BELITTLES

BELITTLING

BELIZE

BELL

BELLA

BELLAMY

BELLATRIX

BELLBOY

BELLBOYS

BELLE

BELLES

BELLEVILLE

BELLHOP

BELLHOPS

BELLICOSE

BELLICOSITY

BELLIES

BELLIGERENCE

BELLIGERENT

BELLIGERENTLY

BELLIGERENTS

BELLINGHAM

BELLINI

BELLMAN

BELLMEN

BELLOVIN

BELLOW

BELLOWED

BELLOWING

BELLOWS

BELLS

BELLUM

BELLWETHER

BELLWETHERS

BELLWOOD

BELLY

BELLYACHE

BEL

BRAKED

BRAKEMAN

BRAKES

BRAKING

BRAMBLE

BRAMBLES

BRAMBLY

BRAN

BRANCH

BRANCHED

BRANCHES

BRANCHING

BRANCHINGS

BRANCHVILLE

BRAND

BRANDED

BRANDEIS

BRANDEL

BRANDENBURG

BRANDING

BRANDISH

BRANDISHES

BRANDISHING

BRANDON

BRANDS

BRANDT

BRANDY

BRANDYWINE

BRANIFF

BRANNON

BRAS

BRASH

BRASHLY

BRASHNESS

BRASILIA

BRASS

BRASSES

BRASSIERE

BRASSTOWN

BRASSY

BRAT

BRATS

BRAUN

BRAVADO

BRAVE

BRAVED

BRAVELY

BRAVENESS

BRAVER

BRAVERY

BRAVES

BRAVEST

BRAVING

BRAVO

BRAVOS

BRAWL

BRAWLER

BRAWLING

BRAWN

BRAY

BRAYED

BRAYER

BRAYING

BRAYS

BRAZE

BRAZED

BRAZEN

BRAZENLY

BRAZENNESS

BRAZES

BRAZIER

BRAZIERS

BRAZIL

BRAZILIAN

BRAZING

BRAZZAVILLE

BREACH

BREACHED

BREACHER

BREACHERS

BREACHES

BREACHING

BREAD

BREADBOARD

BREADBOARDS

BREADBOX

BREADBOXES

BREADED

BREADING

BREADS

BREADTH

BREADWINNER

BREADWINNERS

BREAK

BREAKABLE

BREAKABLES

BREAKAGE

BREAKAWAY

BREAKDOWN

BREAKDOWNS

BREAKER

BREAKERS

BREAKFAST

BREAKFASTED

BREAKFASTER

BREAKFAST

...


SUBSCRIBER

SUBSCRIBERS

SUBSCRIBES

SUBSCRIBING

SUBSCRIPT

SUBSCRIPTED

SUBSCRIPTING

SUBSCRIPTION

SUBSCRIPTIONS

SUBSCRIPTS

SUBSECTION

SUBSECTIONS

SUBSEGMENT

SUBSEGMENTS

SUBSEQUENCE

SUBSEQUENCES

SUBSEQUENT

SUBSEQUENTLY

SUBSERVIENT

SUBSET

SUBSETS

SUBSIDE

SUBSIDED

SUBSIDES

SUBSIDIARIES

SUBSIDIARY

SUBSIDIES

SUBSIDING

SUBSIDIZE

SUBSIDIZED

SUBSIDIZES

SUBSIDIZING

SUBSIDY

SUBSIST

SUBSISTED

SUBSISTENCE

SUBSISTENT

SUBSISTING

SUBSISTS

SUBSLOT

SUBSLOTS

SUBSPACE

SUBSPACES

SUBSTANCE

SUBSTANCES

SUBSTANTIAL

SUBSTANTIALLY

SUBSTANTIATE

SUBSTANTIATED

SUBSTANTIATES

SUBSTANTIATING

SUBSTANTIATION

SUBSTANTIATIONS

SUBSTANTIVE

SUBSTANTIVELY

SUBSTANTIVITY

SUBSTATION

SUBSTATIONS

SUBSTITUTABILITY

SUBSTITUTABLE

SUBSTITUTE

SUBSTITUTED

SUBSTITUTES

SUBSTITUTING

SUBSTITUTION

SUBSTITUTIONS

SUBSTRATE

SUBSTRATES

SUBSTRING

SUBSTRINGS

SUBSTRUCTURE

SUBSTRUCTURES

SUBSUME

SUBSUMED

SUBSUMES

SUBSUMING

SUBSYSTEM

SUBSYSTEMS

SUBTASK

SUBTASKS

SUBTERFUGE

S

TATTOO

TATTOOED

TATTOOS

TAU

TAUGHT

TAUNT

TAUNTED

TAUNTER

TAUNTING

TAUNTS

TAURUS

TAUT

TAUTLY

TAUTNESS

TAUTOLOGICAL

TAUTOLOGICALLY

TAUTOLOGIES

TAUTOLOGY

TAVERN

TAVERNS

TAWNEY

TAWNY

TAX

TAXABLE

TAXATION

TAXED

TAXES

TAXI

TAXICAB

TAXICABS

TAXIED

TAXIING

TAXING

TAXIS

TAXONOMIC

TAXONOMICALLY

TAXONOMY

TAXPAYER

TAXPAYERS

TAYLOR

TAYLORIZE

TAYLORIZES

TAYLORS

TCHAIKOVSKY

TEA

TEACH

TEACHABLE

TEACHER

TEACHERS

TEACHES

TEACHING

TEACHINGS

TEACUP

TEAM

TEAMED

TEAMING

TEAMS

TEAR

TEARED

TEARFUL

TEARFULLY

TEARING

TEARS

TEAS

TEASE

TEASED

TEASES

TEASING

TEASPOON

TEASPOONFUL

TEASPOONFULS

TEASPOONS

TECHNICAL

TECHNICALITIES

TECHNICALITY

TECHNICALLY

TECHNICIAN

TECHNICIANS

TECHNION

TECHNIQUE

TECHNIQUES

TECHNOLOGICAL

TECHNOLOGICALLY

TECHNOLOGIES

TECHNOLOGIST

TECHNOLOGISTS

TECHNOLOGY

TED

TEDDY

TEDIOUS

TEDIOUSLY

TEDIOUSNESS

TEDIUM

TEEM

TEEMED

TEEMING

TEEMS

TEEN

TEENAGE

TEENAGED

TEENAGER

TEENAGERS

TEENS

TEETH

TEETHE

TRICKED

TRICKIER

TRICKIEST

TRICKINESS

TRICKING

TRICKLE

TRICKLED

TRICKLES

TRICKLING

TRICKS

TRICKY

TRIED

TRIER

TRIERS

TRIES

TRIFLE

TRIFLER

TRIFLES

TRIFLING

TRIGGER

TRIGGERED

TRIGGERING

TRIGGERS

TRIGONOMETRIC

TRIGONOMETRY

TRIGRAM

TRIGRAMS

TRIHEDRAL

TRILATERAL

TRILL

TRILLED

TRILLION

TRILLIONS

TRILLIONTH

TRIM

TRIMBLE

TRIMLY

TRIMMED

TRIMMER

TRIMMEST

TRIMMING

TRIMMINGS

TRIMNESS

TRIMS

TRINIDAD

TRINKET

TRINKETS

TRIO

TRIP

TRIPLE

TRIPLED

TRIPLES

TRIPLET

TRIPLETS

TRIPLETT

TRIPLING

TRIPOD

TRIPS

TRISTAN

TRIUMPH

TRIUMPHAL

TRIUMPHANT

TRIUMPHANTLY

TRIUMPHED

TRIUMPHING

TRIUMPHS

TRIVIA

TRIVIAL

TRIVIALITIES

TRIVIALITY

TRIVIALLY

TROBRIAND

TROD

TROJAN

TROLL

TROLLEY

TROLLEYS

TROLLS

TROOP

TROOPER

TROOPERS

TROOPS

TROPEZ

TROPHIES

TROPHY

TROPIC

TROPICAL

TROPICS

TROT

TROTS

TROTSKY

TROUBLE

TROUBLED

TROUBLEMAKER

TROUBLEMAKERS

TROUBLES

TROUBLESHOOT

TROUBLESHOOTER

TROUBLESHOOTERS

TROUBLESHOOTING

TROUBLESHOOTS

TROUBLES


USEFUL

USEFULLY

USEFULNESS

USELESS

USELESSLY

USELESSNESS

USENET

USENIX

USER

USERS

USES

USHER

USHERED

USHERING

USHERS

USING

USUAL

USUALLY

USURP

USURPED

USURPER

UTAH

UTENSIL

UTENSILS

UTICA

UTILITIES

UTILITY

UTILIZATION

UTILIZATIONS

UTILIZE

UTILIZED

UTILIZES

UTILIZING

UTMOST

UTOPIA

UTOPIAN

UTOPIANIZE

UTOPIANIZES

UTOPIANS

UTRECHT

UTTER

UTTERANCE

UTTERANCES

UTTERED

UTTERING

UTTERLY

UTTERMOST

UTTERS

UZI

VACANCIES

VACANCY

VACANT

VACANTLY

VACATE

VACATED

VACATES

VACATING

VACATION

VACATIONED

VACATIONER

VACATIONERS

VACATIONING

VACATIONS

VACUO

VACUOUS

VACUOUSLY

VACUUM

VACUUMED

VACUUMING

VADUZ

VAGABOND

VAGABONDS

VAGARIES

VAGARY

VAGINA

VAGINAS

VAGRANT

VAGRANTLY

VAGUE

VAGUELY

VAGUENESS

VAGUER

VAGUEST

VAIL

VAIN

VAINLY

VALE

VALENCE

VALENCES

VALENTINE

VALENTINES

VALERIE

VALERY

VALES

VALET

VALETS

VALHALLA

VALIANT

VALIANTLY

VALID

VALIDATE

VALIDATED

VALIDATES

VALIDATING

VALIDATION

VALIDITY

VALIDLY

VA

WELCH

WELCHER

WELCHES

WELCOME

WELCOMED

WELCOMES

WELCOMING

WELD

WELDED

WELDER

WELDING

WELDON

WELDS

WELDWOOD

WELFARE

WELL

WELLED

WELLER

WELLES

WELLESLEY

WELLING

WELLINGTON

WELLMAN

WELLS

WELLSVILLE

WELMERS

WELSH

WELTON

WENCH

WENCHES

WENDELL

WENDY

WENT

WENTWORTH

WEPT

WERE

WERNER

WERTHER

WESLEY

WESLEYAN

WESSON

WEST

WESTBOUND

WESTBROOK

WESTCHESTER

WESTERN

WESTERNER

WESTERNERS

WESTFIELD

WESTHAMPTON

WESTINGHOUSE

WESTMINSTER

WESTMORE

WESTON

WESTPHALIA

WESTPORT

WESTWARD

WESTWARDS

WESTWOOD

WET

WETLY

WETNESS

WETS

WETTED

WETTER

WETTEST

WETTING

WEYERHAUSER

WHACK

WHACKED

WHACKING

WHACKS

WHALE

WHALEN

WHALER

WHALES

WHALING

WHARF

WHARTON

WHARVES

WHAT

WHATEVER

WHATLEY

WHATSOEVER

WHEAT

WHEATEN

WHEATLAND

WHEATON

WHEATSTONE

WHEEL

WHEELED

WHEELER

WHEELERS

WHEELING

WHEELINGS

WHEELOCK

WHEELS

WHELAN

WHELLER

WHELP

WHEN

WHENCE

WHENEVER

WHERE

WHEREABOUTS

WHEREAS

WHEREBY

WHEREIN

WHEREUPON

WHEREVER

WHETHER

